In [47]:
import warnings
warnings.filterwarnings("ignore")

In [48]:
import pandas as pd
from darts import TimeSeries
import numpy as np
import matplotlib.pyplot as plt
import logging

logging.disable(logging.CRITICAL)

from darts.dataprocessing.transformers import Scaler
from sklearn.preprocessing import MinMaxScaler
from darts.utils.timeseries_generation import datetime_attribute_timeseries

In [49]:
########### model 1
###########
###########

In [65]:

df_RF_clusted_label_price = pd.read_csv(r'/Users/datle/Desktop/Deakin Uni/5th paper LLM GCN/US DATA/price usa for forcasting.csv')
# df_RF_clusted_label_volume = pd.read_csv(r'/Users/datle/Desktop/Deakin Uni/5th paper LLM GCN/data with cluster/df_RF_volume_filtered.csv')


# df_RF_clusted_label_return = pd.read_csv(r'/Users/datle/Desktop/Deakin Uni/2nd paper code/df_interpolation_clusted_JS_ws36_price_return_3variables_sentiment.csv') # sentiment
# df_RF_clusted_label_return_2 = pd.read_csv(r'/Users/datle/Desktop/Deakin Uni/2nd paper code/df_interpolation_clusted_JS_ws36_price_return_3variables_cpi.csv') #CPI


In [51]:
df_RF_clusted_label_price

,0,1,2,3,4,5,6,7,8,9,...,KMeans,Node2Vec,GAE,GraphSAGE,VGAE,DGI,ARGA,KAN,NO_CLUSTERING,GCN
0,316513,318254,321205,324161,328422,334968,340863,345550,349438,351723,...,3,1,1,1,1,3,1,2,1,1
1,192925,191837,192296,194468,197121,200119,203178,205498,207346,208159,...,2,3,1,1,1,3,1,1,1,0
2,135242,135054,135778,136680,137530,138444,138741,139412,139345,138842,...,2,3,1,1,1,3,1,1,1,0
3,130715,130646,131101,131864,132459,133127,133573,134175,134363,134116,...,2,3,1,1,1,3,1,2,1,0
4,254608,256672,259925,263254,268023,273317,278363,282039,285180,287608,...,3,3,1,1,3,1,1,2,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
296,82283,82242,82278,82261,83100,84393,84940,85162,85957,86126,...,2,1,3,3,1,1,3,1,1,0
297,73399,72994,72811,72934,73840,74114,74499,75215,75662,75780,...,2,3,1,1,3,3,1,2,1,1
298,486760,488880,493409,499302,506420,513294,518556,525454,530375,538250,...,3,1,1,1,3,1,1,2,1,1
299,60083,59905,59970,60480,60966,61212,61639,61689,61613,61686,...,2,3,1,1,1,1,1,2,1,1


In [52]:
def select_cluster_for_loop_testing(price):
    price = price.drop(['Spectral','LabelProp','Hierarchical','HDBSCAN','KMeans','Node2Vec','GAE','GraphSAGE','VGAE','DGI','ARGA','KAN','NO_CLUSTERING','GCN'], axis=1).T
    #  Defining static covariates
    static_covs_multi = pd.DataFrame(data=price.T) # multivariate static covariates (multiple components). note that the number of rows matches the number of components of `series`

    series = TimeSeries.from_times_and_values(
    times=pd.date_range(start="2003-01-31", periods=256, freq="m"),
    values=price, columns = price.columns, static_covariates= static_covs_multi)

    # Create training and validation sets:
    train, val = series.split_after(pd.Timestamp("2019-12-31"))
    # val, test = val.split_after(pd.Timestamp("2022-06-30"))

    # Normalize the time series (note: we avoid fitting the transformer on the validation set)
    scaler = MinMaxScaler(feature_range=(0.001, 1))
    transformer_p = Scaler(scaler)
    train_transformed_p = transformer_p.fit_transform(train)
    val_transformed_p = transformer_p.transform(val)
    # test_transformed_p = transformer_p.transform(test)
    train_val_series_p, test_series = series.split_after(pd.Timestamp("2019-12-31"))
    series_transformed_p = transformer_p.transform(train_val_series_p)

    # create month and year covariate series
    year_series = datetime_attribute_timeseries(
        pd.date_range(start=train_val_series_p.start_time(), freq=train_val_series_p.freq_str, periods=1000),
        attribute="year",
        one_hot=False,
    )
    year_series = Scaler().fit_transform(year_series)
    month_series = datetime_attribute_timeseries(
        year_series, attribute="month", one_hot=True
    )
    covariates = year_series.stack(month_series)
    cov_train, cov_val = covariates.split_after(pd.Timestamp("2019-12-31"))
    
    return covariates, train_transformed_p, val_transformed_p, series_transformed_p, transformer_p # test_transformed_p,

In [53]:
def select_cluster_for_loop_testing_v(volume):
    volume = volume.drop(['Spectral','LabelProp','Hierarchical','HDBSCAN','KMeans','Node2Vec','GAE','GraphSAGE','VGAE','DGI','ARGA','KAN'], axis=1).T
    #  Defining static covariates
    static_covs_multi = pd.DataFrame(data=volume.T) # multivariate static covariates (multiple components). note that the number of rows matches the number of components of `series`

    series = TimeSeries.from_times_and_values(
    times=pd.date_range(start="2003-01-31", periods=256, freq="m"),
    values=volume, columns = volume.columns, static_covariates= static_covs_multi)

    # Create training and validation sets:
    train, val = series.split_after(pd.Timestamp("2019-12-31"))
    # val, test = val.split_after(pd.Timestamp("2022-06-30"))

    # Normalize the time series (note: we avoid fitting the transformer on the validation set)
    scaler = MinMaxScaler(feature_range=(0.001, 1))
    transformer_v = Scaler(scaler)
    train_transformed_v = transformer_v.fit_transform(train)
    val_transformed_v = transformer_v.transform(val)
    # test_transformed_v = transformer_v.transform(test)
    train_val_series_v, test_series = series.split_after(pd.Timestamp("2019-12-31"))
    series_transformed_v = transformer_v.transform(train_val_series_v)

    # create month and year covariate series
    year_series = datetime_attribute_timeseries(
        pd.date_range(start=train_val_series_v.start_time(), freq=train_val_series_v.freq_str, periods=1000),
        attribute="year",
        one_hot=False,
    )
    year_series = Scaler().fit_transform(year_series)
    month_series = datetime_attribute_timeseries(
        year_series, attribute="month", one_hot=True
    )
    covariates = year_series.stack(month_series)
    cov_train, cov_val = covariates.split_after(pd.Timestamp("2019-12-31"))
    
    return covariates, train_transformed_v, val_transformed_v, series_transformed_v, transformer_v #test_transformed_v, 

In [54]:
def select_cluster_for_loop_testing_return(price_return):
    price_return = price_return.drop(['Spectral','LabelProp','Hierarchical','HDBSCAN','KMeans','Node2Vec','GAE','GraphSAGE','VGAE','DGI','ARGA','KAN'], axis=1).T
    #  Defining static covariates
    static_covs_multi = pd.DataFrame(data=price_return.T) # multivariate static covariates (multiple components). note that the number of rows matches the number of components of `series`

    series = TimeSeries.from_times_and_values(
    times=pd.date_range(start="2003-01-31", periods=256, freq="m"),
    values=price_return, columns = price_return.columns, static_covariates= static_covs_multi)

    # Create training and validation sets:
    train, val = series.split_after(pd.Timestamp("2019-12-31"))
    # val, test = val.split_after(pd.Timestamp("2022-06-30"))

    # Normalize the time series (note: we avoid fitting the transformer on the validation set)
    scaler = MinMaxScaler(feature_range=(0.001, 1))
    transformer_r = Scaler(scaler)
    train_transformed_r = transformer_r.fit_transform(train)
    val_transformed_r = transformer_r.transform(val)
    # test_transformed_r = transformer_r.transform(test)
    train_val_series_r, test_series = series.split_after(pd.Timestamp("2019-12-31"))
    series_transformed_r = transformer_r.transform(train_val_series_r)

    # create month and year covariate series
    year_series = datetime_attribute_timeseries(
        pd.date_range(start=train_val_series_r.start_time(), freq=train_val_series_r.freq_str, periods=1000),
        attribute="year",
        one_hot=False,
    )
    year_series = Scaler().fit_transform(year_series)
    month_series = datetime_attribute_timeseries(
        year_series, attribute="month", one_hot=True
    )
    covariates = year_series.stack(month_series)
    cov_train, cov_val = covariates.split_after(pd.Timestamp("2019-12-31"))
    
    return covariates, train_transformed_r, val_transformed_r, series_transformed_r, transformer_r #test_transformed_r, 

In [55]:
def select_cluster_for_loop_testing_return(price_return_2):
    price_return_2 = price_return_2.drop(['Spectral','LabelProp','Hierarchical','HDBSCAN','KMeans','Node2Vec','GAE','GraphSAGE','VGAE','DGI','ARGA','KAN'], axis=1).T
    #  Defining static covariates
    static_covs_multi = pd.DataFrame(data=price_return_2.T) # multivariate static covariates (multiple components). note that the number of rows matches the number of components of `series`

    series = TimeSeries.from_times_and_values(
    times=pd.date_range(start="2003-01-31", periods=256, freq="m"),
    values=price_return_2, columns = price_return_2.columns, static_covariates= static_covs_multi)

    # Create training and validation sets:
    train, val = series.split_after(pd.Timestamp("2019-12-31"))
    # val, test = val.split_after(pd.Timestamp("2022-06-30"))

    # Normalize the time series (note: we avoid fitting the transformer on the validation set)
    scaler = MinMaxScaler(feature_range=(0.001, 1))
    transformer_r2 = Scaler(scaler)
    train_transformed_r2 = transformer_r2.fit_transform(train)
    val_transformed_r2 = transformer_r2.transform(val)
    # test_transformed_r = transformer_r.transform(test)
    train_val_series_r2, test_series = series.split_after(pd.Timestamp("2019-12-31"))
    series_transformed_r2 = transformer_r2.transform(train_val_series_r2)

    # create month and year covariate series
    year_series = datetime_attribute_timeseries(
        pd.date_range(start=train_val_series_r2.start_time(), freq=train_val_series_r2.freq_str, periods=1000),
        attribute="year",
        one_hot=False,
    )
    year_series = Scaler().fit_transform(year_series)
    month_series = datetime_attribute_timeseries(
        year_series, attribute="month", one_hot=True
    )
    covariates = year_series.stack(month_series)
    cov_train, cov_val = covariates.split_after(pd.Timestamp("2019-12-31"))
    
    return covariates, train_transformed_r2, val_transformed_r2, series_transformed_r2, transformer_r2 #test_transformed_r, 

In [56]:
n_epochs = 50
batch_size = 64
random_state = 42
hidden_dim = 20
input_chunk_length= 12
output_chunk_length= 3

In [33]:
# NHITS model
from darts import TimeSeries
from darts.utils.timeseries_generation import datetime_attribute_timeseries
from darts.metrics import mape, rmse, mase, mse, mae, r2_score
from darts.models import NHiTSModel

# We'll define the forecast horizons you want to try
fc_lengths = [6, 12, 24, 36, 48]  # example: a list of multiple n-values

cluster_number = 3
val_result_list = []

for cluster in range(1, cluster_number + 1):
    for cluster_method in [
         'GAE', 'GraphSAGE',
        'VGAE', 'DGI', 'ARGA', 'KAN' #'Spectral', 'LabelProp', 'Hierarchical', 'HDBSCAN', 'KMeans', 'Node2Vec',
    ]:   
        # Select data for this cluster & method
        price = df_RF_clusted_label_price[(df_RF_clusted_label_price[cluster_method] == cluster)]
        # volume = df_RF_clusted_label_volume[(df_RF_clusted_label_volume[cluster_method] == cluster)]

        covariates, train_transformed_p, val_transformed_p, series_transformed_p, transformer_p = select_cluster_for_loop_testing(price)
        # covariates, train_transformed_v, val_transformed_v, series_transformed_v, transformer_v = select_cluster_for_loop_testing(volume)

        # Instantiate the model (once per cluster & method)
        my_model = NHiTSModel(
            input_chunk_length=input_chunk_length,
            output_chunk_length=output_chunk_length,
            optimizer_kwargs={"lr": 1e-3},
            dropout=0,
            n_epochs=n_epochs,
            batch_size=batch_size,
            random_state=random_state,
            save_checkpoints=False,
            force_reset=True,
            pl_trainer_kwargs={"enable_progress_bar": False}  # disable the Lightning progress bar
        )

        # Fit the model
        my_model.fit(
            series=[train_transformed_p],
            verbose=False
        )

        # For each desired forecast length in fc_lengths
        for fc_len in fc_lengths:
            # Predict n=fc_len steps ahead
            pred_series = my_model.predict(n=fc_len, series=train_transformed_p)

            # Evaluate on validation portion
            mape_score = mape(val_transformed_p, pred_series)
            rmse_score = rmse(val_transformed_p, pred_series)*100
            mase_score = mase(val_transformed_p, pred_series, insample=train_transformed_p)
            mse_score = mse(val_transformed_p, pred_series)*100
            mae_score = mae(val_transformed_p, pred_series)*100

            # Append (method, cluster, fc_len, metrics) to the final list
            val_result_list.append([
                cluster_method,
                cluster, 
                fc_len,
                mape_score,
                rmse_score,
                mase_score,
                mse_score,
                mae_score
            ])

# Convert all results to a DataFrame (including fc_len as a column)
val_result = pd.DataFrame(
    val_result_list,
    columns=[
        'Method',      # e.g. 'Spectral'
        'Cluster',     # e.g. 1..3
        'fc_length',   # e.g. 5, 10, 15...
        'mape_val',
        'rmse_val',
        'mase_val',
        'mse_val',
        'mae_val'
    ]
)

print(val_result)
# val_result.to_csv("my_nhits_forecast_results.csv", index=False)

   Method  Cluster  fc_length   mape_val   rmse_val   mase_val    mse_val  \
0     GAE        1          6   3.319332   3.657862   2.105226   0.243961   
1     GAE        1         12   6.586305   9.835740   4.893679   1.533945   
2     GAE        1         24  14.674126  27.259273  14.464068   8.961044   
3     GAE        1         36  20.122685  42.653507  23.635582  21.613363   
4     GAE        1         48  21.719697  47.499257  27.153349  27.688314   
..    ...      ...        ...        ...        ...        ...        ...   
85    KAN        3          6   3.967229   4.206094   3.340167   0.230657   
86    KAN        3         12   7.745677  10.292153   7.165109   1.206250   
87    KAN        3         24  18.527974  31.872390  21.366587  11.123306   
88    KAN        3         36  26.373033  49.325023  34.361719  26.048826   
89    KAN        3         48  30.338384  55.611490  40.523534  32.990302   

      mae_val  
0    3.228278  
1    7.687469  
2   21.680209  
3   35.0417

In [31]:
# Suppose each row in val_result_list_v is like:
# [Method, Cluster, fc_len, mape_val, rmse_val, mase_val, mse_val, mae_val]

df_val = pd.DataFrame(
    val_result_list,
    columns=[
        "Method",    # e.g. 'Spectral'
        "Cluster",   # 1..3
        "fc_len",    # forecast length, e.g. 5, 10, 20, etc.
        "mape_val",
        "rmse_val",
        "mase_val",
        "mse_val",
        "mae_val"
    ]
)

# We'll find all unique forecast lengths in df_val['fc_len']
unique_fc_lens = sorted(df_val["fc_len"].unique())

for fc in unique_fc_lens:
    print(f"\n=== fc_len = {fc} ===")
    
    # 1) Filter rows for this forecast length
    df_subset = df_val[df_val["fc_len"] == fc]
    
    # 2) Group by 'Method' and compute mean of each metric
    method_means = (
        df_subset
        .groupby("Method", as_index=False)[["mape_val", "rmse_val", "mase_val", "mse_val", "mae_val"]]
        .mean()
    )
    
    # 3) Print summary table for this fc_len
    print("\n--- Mean of each metric per Method ---")
    print(method_means)
    
    # (Optional) Overall mean across all methods for this fc_len
    overall_means = df_subset[["mape_val", "rmse_val", "mase_val", "mse_val", "mae_val"]].mean()
    print("\n--- Overall Mean of each metric (all methods) for fc_len:", fc, "---")
    print(overall_means)


=== fc_len = 6 ===

--- Mean of each metric per Method ---
      Method   mape_val   rmse_val  mase_val   mse_val    mae_val
0       ARGA   8.980083   9.550861  5.150998  1.201346   8.555313
1        DGI  10.852888  11.434557  6.992711  1.715080  10.293263
2        GAE   8.814785   9.474357  4.988501  1.186932   8.413605
3  GraphSAGE   9.157678   9.700672  5.172288  1.227058   8.653658
4        KAN   8.824708   9.438993  6.069613  1.276312   8.520216
5       VGAE   8.906699   9.574302  5.204613  1.221674   8.545071

--- Overall Mean of each metric (all methods) for fc_len: 6 ---
mape_val    9.256140
rmse_val    9.862290
mase_val    5.596454
mse_val     1.304734
mae_val     8.830188
dtype: float64

=== fc_len = 12 ===

--- Mean of each metric per Method ---
      Method   mape_val   rmse_val  mase_val   mse_val    mae_val
0       ARGA  13.092362  18.550932  8.339329  4.235624  15.419864
1        DGI  12.607206  15.916093  9.043467  3.477456  13.817388
2        GAE  13.025752  18.694177

In [ ]:
# ONLY FOR KAN TO COMPARE, HENCE FIND THE BEST PARAMETERS
# ONLY FOR KAN TO COMPARE, HENCE FIND THE BEST PARAMETERS
# ONLY FOR KAN TO COMPARE, HENCE FIND THE BEST PARAMETERS
# ONLY FOR KAN TO COMPARE, HENCE FIND THE BEST PARAMETERS
# ONLY FOR KAN TO COMPARE, HENCE FIND THE BEST PARAMETERS


In [14]:
# NHITS model
from darts import TimeSeries
from darts.utils.timeseries_generation import datetime_attribute_timeseries
from darts.metrics import mape, rmse, mase, mse, mae, r2_score
from darts.models import NHiTSModel

# We'll define the forecast horizons you want to try
fc_lengths = [6, 12, 24, 36, 40]  # example: a list of multiple n-values

cluster_number = 3
val_result_list = []

for cluster in range(1, cluster_number + 1):
    for cluster_method in ['KAN']:
        # Select data for this cluster & method
        price = df_RF_clusted_label_price[(df_RF_clusted_label_price[cluster_method] == cluster)]
        covariates, train_transformed_p, val_transformed_p, series_transformed_p, transformer_p = select_cluster_for_loop_testing(price)

        # Instantiate the model (once per cluster & method)
        my_model = NHiTSModel(
            input_chunk_length=input_chunk_length,
            output_chunk_length=output_chunk_length,
            optimizer_kwargs={"lr": 1e-3},
            dropout=0,
            n_epochs=100,
            batch_size=batch_size,
            random_state=random_state,
            save_checkpoints=False,
            force_reset=True,
            pl_trainer_kwargs={"enable_progress_bar": False}  # disable the Lightning progress bar
        )

        # Fit the model
        my_model.fit(
            series=[train_transformed_p],
            verbose=False
        )

        # For each desired forecast length in fc_lengths
        for fc_len in fc_lengths:
            # Predict n=fc_len steps ahead
            pred_series = my_model.predict(n=fc_len, series=train_transformed_p)

            # Evaluate on validation portion
            mape_score = mape(val_transformed_p, pred_series)
            rmse_score = rmse(val_transformed_p, pred_series)*100
            mase_score = mase(val_transformed_p, pred_series, insample=train_transformed_p)
            mse_score = mse(val_transformed_p, pred_series)*100
            mae_score = mae(val_transformed_p, pred_series)*100

            # Append (method, cluster, fc_len, metrics) to the final list
            val_result_list.append([
                cluster_method,
                cluster, 
                fc_len,
                mape_score,
                rmse_score,
                mase_score,
                mse_score,
                mae_score
            ])

# Convert all results to a DataFrame (including fc_len as a column)
val_result = pd.DataFrame(
    val_result_list,
    columns=[
        'Method',      # e.g. 'Spectral'
        'Cluster',     # e.g. 1..3
        'fc_length',   # e.g. 5, 10, 15...
        'mape_val',
        'rmse_val',
        'mase_val',
        'mse_val',
        'mae_val'
    ]
)

print(val_result)
# val_result.to_csv("my_nhits_forecast_results.csv", index=False)

   Method  Cluster  fc_length   mape_val   rmse_val   mase_val    mse_val  \
0     KAN        1          6   9.580723   8.851086   5.038064   1.201367   
1     KAN        1         12  13.774880  14.050866   7.999608   2.795495   
2     KAN        1         24  19.634377  24.367565  14.133878   7.937007   
3     KAN        1         36  22.006377  29.051388  16.997239  10.997421   
4     KAN        1         40  22.657820  30.718658  17.978311  12.183697   
5     KAN        2          6   8.930873   8.720003   4.818256   1.191191   
6     KAN        2         12  11.968458  13.769698   7.475105   2.964997   
7     KAN        2         24  17.010304  23.584626  13.082003   7.713806   
8     KAN        2         36  18.786507  26.867587  14.908697   9.677334   
9     KAN        2         40  18.963952  27.414274  15.267244  10.052878   
10    KAN        3          6  12.960039   8.575172   4.427294   1.156150   
11    KAN        3         12  15.538178  12.229125   6.248214   2.072518   

In [15]:
# Suppose each row in val_result_list_v is like:
# [Method, Cluster, fc_len, mape_val, rmse_val, mase_val, mse_val, mae_val]

df_val = pd.DataFrame(
    val_result_list,
    columns=[
        "Method",    # e.g. 'Spectral'
        "Cluster",   # 1..3
        "fc_len",    # forecast length, e.g. 5, 10, 20, etc.
        "mape_val",
        "rmse_val",
        "mase_val",
        "mse_val",
        "mae_val"
    ]
)

# We'll find all unique forecast lengths in df_val['fc_len']
unique_fc_lens = sorted(df_val["fc_len"].unique())

for fc in unique_fc_lens:
    print(f"\n=== fc_len = {fc} ===")
    
    # 1) Filter rows for this forecast length
    df_subset = df_val[df_val["fc_len"] == fc]
    
    # 2) Group by 'Method' and compute mean of each metric
    method_means = (
        df_subset
        .groupby("Method", as_index=False)[["mape_val", "rmse_val", "mase_val", "mse_val", "mae_val"]]
        .mean()
    )
    
    # 3) Print summary table for this fc_len
    print("\n--- Mean of each metric per Method ---")
    print(method_means)
    
    # (Optional) Overall mean across all methods for this fc_len
    overall_means = df_subset[["mape_val", "rmse_val", "mase_val", "mse_val", "mae_val"]].mean()
    print("\n--- Overall Mean of each metric (all methods) for fc_len:", fc, "---")
    print(overall_means)


=== fc_len = 6 ===

--- Mean of each metric per Method ---
  Method   mape_val  rmse_val  mase_val   mse_val   mae_val
0    KAN  10.490545   8.71542  4.761205  1.182902  7.820116

--- Overall Mean of each metric (all methods) for fc_len: 6 ---
mape_val    10.490545
rmse_val     8.715420
mase_val     4.761205
mse_val      1.182902
mae_val      7.820116
dtype: float64

=== fc_len = 12 ===

--- Mean of each metric per Method ---
  Method   mape_val   rmse_val  mase_val   mse_val    mae_val
0    KAN  13.760506  13.349896  7.240976  2.611004  11.553737

--- Overall Mean of each metric (all methods) for fc_len: 12 ---
mape_val    13.760506
rmse_val    13.349896
mase_val     7.240976
mse_val      2.611004
mae_val     11.553737
dtype: float64

=== fc_len = 24 ===

--- Mean of each metric per Method ---
  Method   mape_val   rmse_val   mase_val  mse_val    mae_val
0    KAN  18.666408  21.985532  12.227081  6.71077  18.920812

--- Overall Mean of each metric (all methods) for fc_len: 24 ---
map

In [39]:
n_epochs = 10
batch_size = 32
random_state = 42
hidden_dim = 20
input_chunk_length= 12
output_chunk_length= 3

In [ ]:
# TSMixerModel model
from darts import TimeSeries
from darts.utils.timeseries_generation import datetime_attribute_timeseries
from darts.metrics import mape, rmse, mase, mse, mae, r2_score
from darts.models import TSMixerModel

# We'll define the forecast horizons you want to try
fc_lengths = [6, 12, 24, 36, 48]  # example: a list of multiple n-values

cluster_number = 3
val_result_list = []

for cluster in range(1, cluster_number + 1):
    for cluster_method in [
       'GAE', 'GraphSAGE',
        'VGAE', 'DGI', 'ARGA', 'KAN' # 'Spectral', 'LabelProp', 'Hierarchical', 'HDBSCAN',  'KMeans', 'Node2Vec', 
      
    ]:
        # Select data for this cluster & method
        price = df_RF_clusted_label_price[(df_RF_clusted_label_price[cluster_method] == cluster)]
        covariates, train_transformed_p, val_transformed_p, series_transformed_p, transformer_p = select_cluster_for_loop_testing(price)

        # Instantiate the model (once per cluster & method)
        my_model = TSMixerModel(
            input_chunk_length=input_chunk_length,
            output_chunk_length=output_chunk_length,
            optimizer_kwargs={"lr": 1e-3},
            dropout=0,
            n_epochs=n_epochs,
            batch_size=batch_size,
            random_state=random_state,
            save_checkpoints=False,
            force_reset=True,
            pl_trainer_kwargs={"enable_progress_bar": False}  # disable the Lightning progress bar
        )

        # Fit the model
        my_model.fit(
            series=[train_transformed_p],
            verbose=False
        )

        # For each desired forecast length in fc_lengths
        for fc_len in fc_lengths:
            # Predict n=fc_len steps ahead
            pred_series = my_model.predict(n=fc_len, series=train_transformed_p)

            # Evaluate on validation portion
            mape_score = mape(val_transformed_p, pred_series)
            rmse_score = rmse(val_transformed_p, pred_series)*100
            mase_score = mase(val_transformed_p, pred_series, insample=train_transformed_p)
            mse_score = mse(val_transformed_p, pred_series)*100
            mae_score = mae(val_transformed_p, pred_series)*100

            # Append (method, cluster, fc_len, metrics) to the final list
            val_result_list.append([
                cluster_method,
                cluster, 
                fc_len,
                mape_score,
                rmse_score,
                mase_score,
                mse_score,
                mae_score
            ])

# Convert all results to a DataFrame (including fc_len as a column)
val_result = pd.DataFrame(
    val_result_list,
    columns=[
        'Method',      # e.g. 'Spectral'
        'Cluster',     # e.g. 1..3
        'fc_length',   # e.g. 5, 10, 15...
        'mape_val',
        'rmse_val',
        'mase_val',
        'mse_val',
        'mae_val'
    ]
)

print(val_result)
# val_result.to_csv("my_nhits_forecast_results.csv", index=False)

   Method  Cluster  fc_length   mape_val    rmse_val   mase_val     mse_val  \
0     GAE        1          6  46.398144   47.252476  34.305753   24.828514   
1     GAE        1         12  50.999496   58.485220  41.066011   37.314018   
2     GAE        1         24  58.506502   83.676616  56.551029   74.235640   
3     GAE        1         36  63.446245  107.439435  71.291980  121.634957   
4     GAE        1         48  66.282392  121.282883  80.503289  155.067051   
..    ...      ...        ...        ...         ...        ...         ...   
85    KAN        3          6  26.052389   26.643386  23.218269    8.949092   
86    KAN        3         12  30.776616   34.452045  29.131125   14.100842   
87    KAN        3         24  40.604714   57.765692  45.781687   37.337371   
88    KAN        3         36  46.789760   75.763125  59.525239   62.824627   
89    KAN        3         48  49.651410   82.165296  65.595258   73.568112   

       mae_val  
0    46.787779  
1    57.000922  


In [36]:
# Suppose each row in val_result_list_v is like:
# [Method, Cluster, fc_len, mape_val, rmse_val, mase_val, mse_val, mae_val]

df_val = pd.DataFrame(
    val_result_list,
    columns=[
        "Method",    # e.g. 'Spectral'
        "Cluster",   # 1..3
        "fc_len",    # forecast length, e.g. 5, 10, 20, etc.
        "mape_val",
        "rmse_val",
        "mase_val",
        "mse_val",
        "mae_val"
    ]
)

# We'll find all unique forecast lengths in df_val['fc_len']
unique_fc_lens = sorted(df_val["fc_len"].unique())

for fc in unique_fc_lens:
    print(f"\n=== fc_len = {fc} ===")
    
    # 1) Filter rows for this forecast length
    df_subset = df_val[df_val["fc_len"] == fc]
    
    # 2) Group by 'Method' and compute mean of each metric
    method_means = (
        df_subset
        .groupby("Method", as_index=False)[["mape_val", "rmse_val", "mase_val", "mse_val", "mae_val"]]
        .mean()
    )
    
    # 3) Print summary table for this fc_len
    print("\n--- Mean of each metric per Method ---")
    print(method_means)
    
    # (Optional) Overall mean across all methods for this fc_len
    overall_means = df_subset[["mape_val", "rmse_val", "mase_val", "mse_val", "mae_val"]].mean()
    print("\n--- Overall Mean of each metric (all methods) for fc_len:", fc, "---")
    print(overall_means)


=== fc_len = 6 ===

--- Mean of each metric per Method ---
      Method   mape_val   rmse_val  mase_val   mse_val    mae_val
0       ARGA   9.945049  11.269686  5.918096  1.654286   9.965575
1        DGI  10.552846  11.369917  7.192308  1.751600  10.516186
2        GAE   9.957763  11.236622  5.976006  1.640037   9.951288
3  GraphSAGE  10.131718  11.409798  6.012483  1.702270  10.108412
4        KAN  10.841896  11.650883  7.588517  1.914208  10.853275
5       VGAE   9.577856  11.011536  5.768371  1.550629   9.623068

--- Overall Mean of each metric (all methods) for fc_len: 6 ---
mape_val    10.167854
rmse_val    11.324740
mase_val     6.409297
mse_val      1.702172
mae_val     10.169634
dtype: float64

=== fc_len = 12 ===

--- Mean of each metric per Method ---
      Method   mape_val   rmse_val   mase_val   mse_val    mae_val
0       ARGA  18.339905  26.571100  12.137053  8.338149  22.241614
1        DGI  16.652115  22.057727  12.803658  6.198306  19.242849
2        GAE  18.175857  2

In [45]:

n_epochs = 100
batch_size = 32
random_state = 42
hidden_dim = 20
input_chunk_length= 12
output_chunk_length= 3

# TransformerModel model
from darts import TimeSeries
from darts.utils.timeseries_generation import datetime_attribute_timeseries
from darts.metrics import mape, rmse, mase, mse, mae, r2_score
from darts.models import TransformerModel

# We'll define the forecast horizons you want to try
fc_lengths = [6, 12, 24, 36, 48]  # example: a list of multiple n-values

cluster_number = 3
val_result_list = []

for cluster in range(1, cluster_number + 1):
    for cluster_method in [
       'GAE', 'GraphSAGE',
        'VGAE', 'DGI', 'ARGA', 'KAN' # 'Spectral', 'LabelProp', 'Hierarchical', 'HDBSCAN',  'KMeans', 'Node2Vec', 
      
    ]:
        # Select data for this cluster & method
        price = df_RF_clusted_label_price[(df_RF_clusted_label_price[cluster_method] == cluster)]
        covariates, train_transformed_p, val_transformed_p, series_transformed_p, transformer_p = select_cluster_for_loop_testing(price)

        # Instantiate the model (once per cluster & method)
        my_model = TransformerModel(
            input_chunk_length=input_chunk_length,
            output_chunk_length=output_chunk_length,
            optimizer_kwargs={"lr": 1e-3},
            dropout=0,
            n_epochs=n_epochs,
            batch_size=batch_size,
            random_state=random_state,
            save_checkpoints=False,
            force_reset=True,
            pl_trainer_kwargs={"enable_progress_bar": False}  # disable the Lightning progress bar
        )

        # Fit the model
        my_model.fit(
            series=[train_transformed_p],
            verbose=False
        )

        # For each desired forecast length in fc_lengths
        for fc_len in fc_lengths:
            # Predict n=fc_len steps ahead
            pred_series = my_model.predict(n=fc_len, series=train_transformed_p)

            # Evaluate on validation portion
            mape_score = mape(val_transformed_p, pred_series)
            rmse_score = rmse(val_transformed_p, pred_series)*100
            mase_score = mase(val_transformed_p, pred_series, insample=train_transformed_p)
            mse_score = mse(val_transformed_p, pred_series)*100
            mae_score = mae(val_transformed_p, pred_series)*100

            # Append (method, cluster, fc_len, metrics) to the final list
            val_result_list.append([
                cluster_method,
                cluster, 
                fc_len,
                mape_score,
                rmse_score,
                mase_score,
                mse_score,
                mae_score
            ])

# Convert all results to a DataFrame (including fc_len as a column)
val_result = pd.DataFrame(
    val_result_list,
    columns=[
        'Method',      # e.g. 'Spectral'
        'Cluster',     # e.g. 1..3
        'fc_length',   # e.g. 5, 10, 15...
        'mape_val',
        'rmse_val',
        'mase_val',
        'mse_val',
        'mae_val'
    ]
)

print(val_result)
# val_result.to_csv("my_nhits_forecast_results.csv", index=False)

   Method  Cluster  fc_length   mape_val   rmse_val   mase_val    mse_val  \
0     GAE        1          6   7.444354   8.485625   5.070747   0.981907   
1     GAE        1         12  13.755952  19.035703  10.655425   4.616286   
2     GAE        1         24  25.172303  43.876901  24.877380  21.384169   
3     GAE        1         36  33.386194  67.396871  39.109910  49.681185   
4     GAE        1         48  38.117293  80.483744  48.059742  71.121879   
..    ...      ...        ...        ...        ...        ...        ...   
85    KAN        3          6   5.666849   6.064206   4.653001   0.424308   
86    KAN        3         12  10.569455  13.445170   9.524210   1.954522   
87    KAN        3         24  21.855996  36.000343  24.662115  13.962904   
88    KAN        3         36  29.494054  53.415079  37.825153  30.351713   
89    KAN        3         48  32.985485  59.085259  43.603158  37.145100   

      mae_val  
0    7.480424  
1   16.020592  
2   36.528616  
3   56.9508

In [46]:
# Suppose each row in val_result_list_v is like:
# [Method, Cluster, fc_len, mape_val, rmse_val, mase_val, mse_val, mae_val]

df_val = pd.DataFrame(
    val_result_list,
    columns=[
        "Method",    # e.g. 'Spectral'
        "Cluster",   # 1..3
        "fc_len",    # forecast length, e.g. 5, 10, 20, etc.
        "mape_val",
        "rmse_val",
        "mase_val",
        "mse_val",
        "mae_val"
    ]
)

# We'll find all unique forecast lengths in df_val['fc_len']
unique_fc_lens = sorted(df_val["fc_len"].unique())

for fc in unique_fc_lens:
    print(f"\n=== fc_len = {fc} ===")
    
    # 1) Filter rows for this forecast length
    df_subset = df_val[df_val["fc_len"] == fc]
    
    # 2) Group by 'Method' and compute mean of each metric
    method_means = (
        df_subset
        .groupby("Method", as_index=False)[["mape_val", "rmse_val", "mase_val", "mse_val", "mae_val"]]
        .mean()
    )
    
    # 3) Print summary table for this fc_len
    print("\n--- Mean of each metric per Method ---")
    print(method_means)
    
    # (Optional) Overall mean across all methods for this fc_len
    overall_means = df_subset[["mape_val", "rmse_val", "mase_val", "mse_val", "mae_val"]].mean()
    print("\n--- Overall Mean of each metric (all methods) for fc_len:", fc, "---")
    print(overall_means)


=== fc_len = 6 ===

--- Mean of each metric per Method ---
      Method  mape_val  rmse_val  mase_val   mse_val   mae_val
0       ARGA  6.337265  7.003364  3.674514  0.645362  6.195225
1        DGI  6.878628  7.765361  4.630676  0.796510  6.823099
2        GAE  6.259098  6.974078  3.659261  0.646709  6.159071
3  GraphSAGE  7.811348  8.325018  4.770669  0.934247  7.649349
4        KAN  6.639486  7.421119  4.602050  0.737115  6.548400
5       VGAE  7.310681  7.996256  4.540050  0.853218  7.248180

--- Overall Mean of each metric (all methods) for fc_len: 6 ---
mape_val    6.872751
rmse_val    7.580866
mase_val    4.312870
mse_val     0.768860
mae_val     6.770554
dtype: float64

=== fc_len = 12 ===

--- Mean of each metric per Method ---
      Method   mape_val   rmse_val   mase_val   mse_val    mae_val
0       ARGA  13.091878  19.686946   8.771663  4.583232  15.806491
1        DGI  13.062380  18.250195  10.035468  4.075240  15.149373
2        GAE  13.110573  19.791900   8.829044  4.648

In [63]:
###### NO CLUSTERING

n_epochs = 10
batch_size = 64
random_state = 42
hidden_dim = 20
input_chunk_length= 12
output_chunk_length= 3

# NHITS/ TSMixer model/ Tranf
from darts import TimeSeries
from darts.utils.timeseries_generation import datetime_attribute_timeseries
from darts.metrics import mape, rmse, mase, mse, mae, r2_score
from darts.models import NHiTSModel
from darts.models import TSMixerModel
from darts.models import TransformerModel


# Check if MPS is available

# We'll define the forecast horizons you want to try
fc_lengths = [6, 12, 24, 36, 48]  # example: a list of multiple n-values

cluster_number = 1
val_result_list = []

for cluster in range(1, cluster_number + 1):
    for cluster_method in [
        'NO_CLUSTERING'# 'Spectral', 'LabelProp', 'Hierarchical', 'HDBSCAN', 'KMeans', 'Node2Vec', GAE', 'GraphSAGE','VGAE', 'DGI', 'ARGA', 'KAN' 
    ]:
        # Select data for this cluster & method
        price = df_RF_clusted_label_price[(df_RF_clusted_label_price[cluster_method] == cluster)]
        # volume = df_RF_clusted_label_volume[(df_RF_clusted_label_volume[cluster_method] == cluster)]

        covariates, train_transformed_p, val_transformed_p, series_transformed_p, transformer_p = select_cluster_for_loop_testing(price)
        # covariates, train_transformed_v, val_transformed_v, series_transformed_v, transformer_v = select_cluster_for_loop_testing(volume)


        # Instantiate the model (once per cluster & method)
        my_model = NHiTSModel(
            input_chunk_length=input_chunk_length,
            output_chunk_length=output_chunk_length,
            optimizer_kwargs={"lr": 1e-3},
            dropout=0,
            n_epochs=n_epochs,
            batch_size=batch_size,
            random_state=random_state,
            save_checkpoints=False,
            force_reset=True,
            pl_trainer_kwargs={
                "enable_progress_bar": False,  # Disable the progress bar
            }  
        )

        # Fit the model
        my_model.fit(
            series=[train_transformed_p], #train_transformed_p train_transformed_v
            verbose=False
        )

        # For each desired forecast length in fc_lengths
        for fc_len in fc_lengths:
            # Predict n=fc_len steps ahead
            pred_series = my_model.predict(n=fc_len, series=train_transformed_p)

            # Evaluate on validation portion
            mape_score = mape(val_transformed_p, pred_series)
            rmse_score = rmse(val_transformed_p, pred_series)*100
            mase_score = mase(val_transformed_p, pred_series, insample=train_transformed_p)
            mse_score = mse(val_transformed_p, pred_series)*100
            mae_score = mae(val_transformed_p, pred_series)*100

            # Append (method, cluster, fc_len, metrics) to the final list
            val_result_list.append([
                cluster_method,
                cluster, 
                fc_len,
                mape_score,
                rmse_score,
                mase_score,
                mse_score,
                mae_score
            ])

# Convert all results to a DataFrame (including fc_len as a column)
val_result = pd.DataFrame(
    val_result_list,
    columns=[
        'Method',      # e.g. 'Spectral'
        'Cluster',     # e.g. 1..3
        'fc_length',   # e.g. 5, 10, 15...
        'mape_val',
        'rmse_val',
        'mase_val',
        'mse_val',
        'mae_val'
    ]
)

print(val_result)
# val_result.to_csv("my_nhits_forecast_results.csv", index=False)

          Method  Cluster  fc_length   mape_val   rmse_val   mase_val  \
0  NO_CLUSTERING        1          6   9.731806  10.481893   6.342433   
1  NO_CLUSTERING        1         12  13.137206  17.314730   9.659082   
2  NO_CLUSTERING        1         24  20.829261  35.740648  19.693641   
3  NO_CLUSTERING        1         36  26.737724  53.736820  30.193102   
4  NO_CLUSTERING        1         48  29.303241  61.494267  35.516204   

     mse_val    mae_val  
0   1.526526   9.454190  
1   4.317931  14.967576  
2  16.156732  30.068899  
3  35.304964  45.597180  
4  46.927211  53.876132  


In [72]:
###### GCN CLUSTERING

n_epochs = 50
batch_size = 64
random_state = 42
hidden_dim = 20
input_chunk_length= 12
output_chunk_length= 3

# NHITS/ TSMixer model/ Tranf
from darts import TimeSeries
from darts.utils.timeseries_generation import datetime_attribute_timeseries
from darts.metrics import mape, rmse, mase, mse, mae, r2_score
from darts.models import NHiTSModel
from darts.models import TSMixerModel
from darts.models import TransformerModel


# Check if MPS is available

# We'll define the forecast horizons you want to try
fc_lengths = [6, 12, 24, 36, 48]  # example: a list of multiple n-values

cluster_number = 3
val_result_list = []

for cluster in range(1, cluster_number + 1):
    for cluster_method in [
        'GCN'# 'Spectral', 'LabelProp', 'Hierarchical', 'HDBSCAN', 'KMeans', 'Node2Vec', GAE', 'GraphSAGE','VGAE', 'DGI', 'ARGA', 'KAN' 
    ]:
        # Select data for this cluster & method
        price = df_RF_clusted_label_price[(df_RF_clusted_label_price[cluster_method] == cluster)]
        # volume = df_RF_clusted_label_volume[(df_RF_clusted_label_volume[cluster_method] == cluster)]

        covariates, train_transformed_p, val_transformed_p, series_transformed_p, transformer_p = select_cluster_for_loop_testing(price)
        # covariates, train_transformed_v, val_transformed_v, series_transformed_v, transformer_v = select_cluster_for_loop_testing(volume)


        # Instantiate the model (once per cluster & method)
        my_model = TransformerModel(
            input_chunk_length=input_chunk_length,
            output_chunk_length=output_chunk_length,
            optimizer_kwargs={"lr": 1e-3},
            dropout=0,
            n_epochs=n_epochs,
            batch_size=batch_size,
            random_state=random_state,
            save_checkpoints=False,
            force_reset=True,
            pl_trainer_kwargs={
                "enable_progress_bar": False,  # Disable the progress bar
            }  
        )

        # Fit the model
        my_model.fit(
            series=[train_transformed_p], #train_transformed_p train_transformed_v
            verbose=False
        )

        # For each desired forecast length in fc_lengths
        for fc_len in fc_lengths:
            # Predict n=fc_len steps ahead
            pred_series = my_model.predict(n=fc_len, series=train_transformed_p)

            # Evaluate on validation portion
            mape_score = mape(val_transformed_p, pred_series)
            rmse_score = rmse(val_transformed_p, pred_series)*100
            mase_score = mase(val_transformed_p, pred_series, insample=train_transformed_p)
            mse_score = mse(val_transformed_p, pred_series)*100
            mae_score = mae(val_transformed_p, pred_series)*100

            # Append (method, cluster, fc_len, metrics) to the final list
            val_result_list.append([
                cluster_method,
                cluster, 
                fc_len,
                mape_score,
                rmse_score,
                mase_score,
                mse_score,
                mae_score
            ])

# Convert all results to a DataFrame (including fc_len as a column)
val_result = pd.DataFrame(
    val_result_list,
    columns=[
        'Method',      # e.g. 'Spectral'
        'Cluster',     # e.g. 1..3
        'fc_length',   # e.g. 5, 10, 15...
        'mape_val',
        'rmse_val',
        'mase_val',
        'mse_val',
        'mae_val'
    ]
)

# print(val_result)
# # val_result.to_csv("my_nhits_forecast_results.csv", index=False)

# Suppose each row in val_result_list_v is like:
# [Method, Cluster, fc_len, mape_val, rmse_val, mase_val, mse_val, mae_val]

df_val = pd.DataFrame(
    val_result_list,
    columns=[
        "Method",    # e.g. 'Spectral'
        "Cluster",   # 1..3
        "fc_len",    # forecast length, e.g. 5, 10, 20, etc.
        "mape_val",
        "rmse_val",
        "mase_val",
        "mse_val",
        "mae_val"
    ]
)

# We'll find all unique forecast lengths in df_val['fc_len']
unique_fc_lens = sorted(df_val["fc_len"].unique())

for fc in unique_fc_lens:
    print(f"\n=== fc_len = {fc} ===")
    
    # 1) Filter rows for this forecast length
    df_subset = df_val[df_val["fc_len"] == fc]
    
    # 2) Group by 'Method' and compute mean of each metric
    method_means = (
        df_subset
        .groupby("Method", as_index=False)[["mape_val", "rmse_val", "mase_val", "mse_val", "mae_val"]]
        .mean()
    )
    
    # 3) Print summary table for this fc_len
    print("\n--- Mean of each metric per Method ---")
    print(method_means)
    
    # # (Optional) Overall mean across all methods for this fc_len
    # overall_means = df_subset[["mape_val", "rmse_val", "mase_val", "mse_val", "mae_val"]].mean()
    # print("\n--- Overall Mean of each metric (all methods) for fc_len:", fc, "---")
    # print(overall_means)


=== fc_len = 6 ===

--- Mean of each metric per Method ---
  Method  mape_val   rmse_val  mase_val  mse_val   mae_val
0    GCN  9.861827  10.758664  7.039139  1.55019  9.726453

=== fc_len = 12 ===

--- Mean of each metric per Method ---
  Method   mape_val   rmse_val  mase_val   mse_val    mae_val
0    GCN  15.526393  20.317969  12.44764  5.322794  17.639896

=== fc_len = 24 ===

--- Mean of each metric per Method ---
  Method   mape_val   rmse_val   mase_val    mse_val    mae_val
0    GCN  26.178996  43.651349  26.495698  21.665643  36.930897

=== fc_len = 36 ===

--- Mean of each metric per Method ---
  Method   mape_val   rmse_val   mase_val    mse_val    mae_val
0    GCN  33.966421  64.985821  40.134639  46.977083  55.611642

=== fc_len = 48 ===

--- Mean of each metric per Method ---
  Method   mape_val   rmse_val   mase_val    mse_val    mae_val
0    GCN  38.327374  75.877416  48.020283  64.707598  66.824395
